In [1]:
# Import everything we'll need

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms, models
import timm
import os
import zipfile
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import Image
from pathlib import Path

In [2]:
# Load our data
from google.colab import files
uploaded = files.upload()

Saving data.zip to data.zip


In [5]:
with zipfile.ZipFile('data.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [6]:
# Path to the directory with PNG files
input_dir = Path('data/fake')

# Iterate through each file in the directory
for file_path in input_dir.iterdir():
    # Check if it's a file and ends with .png
    if file_path.is_file() and file_path.suffix.lower() == '.png':
        try:
            # Open and convert to RGB (JPEG doesn’t support transparency)
            with Image.open(file_path) as img:
                rgb_img = img.convert('RGB')
                jpg_path = file_path.with_suffix('.jpg')
                rgb_img.save(jpg_path, 'JPEG')

            # Delete the original PNG
            file_path.unlink()

            print(f"Converted and deleted: {file_path.name}")

        except Exception as e:
            print(f"Error processing {file_path.name}: {e}")

Converted and deleted: 30.png
Converted and deleted: 40.png
Converted and deleted: 8.png
Converted and deleted: 2.png
Converted and deleted: 16.png
Converted and deleted: 36.png
Converted and deleted: 24.png
Converted and deleted: 14.png
Converted and deleted: 13.png
Converted and deleted: 37.png
Converted and deleted: 31.png
Converted and deleted: 20.png
Converted and deleted: 38.png
Converted and deleted: 18.png
Converted and deleted: 28.png
Converted and deleted: 11.png
Converted and deleted: 15.png
Converted and deleted: 6.png
Converted and deleted: 32.png
Converted and deleted: 19.png
Converted and deleted: 39.png
Converted and deleted: 9.png
Converted and deleted: 10.png
Converted and deleted: 4.png
Converted and deleted: 1.png
Converted and deleted: 34.png
Converted and deleted: 5.png
Converted and deleted: 26.png
Converted and deleted: 3.png
Converted and deleted: 33.png
Converted and deleted: 27.png
Converted and deleted: 21.png
Converted and deleted: 12.png
Converted and dele

In [8]:
full_dataset = datasets.ImageFolder('data')

# Extract labels for stratification
targets = [label for _, label in full_dataset]

In [9]:
# Stratified split

train_idx, val_idx = train_test_split(
    np.arange(len(targets)),
    test_size=0.2,
    stratify=targets,
    random_state=42
)

# Create subset datasets
train_subset = Subset(full_dataset, train_idx)
val_subset = Subset(full_dataset, val_idx)

In [10]:
# Define transforms to pre-process and augment our training data since for logistical reason we only have a very small number of samples to work with

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),     # Random zoom/crop
    transforms.RandomHorizontalFlip(),                       # Flip image
    transforms.RandomRotation(degrees=10),                   # Small rotations
    transforms.ColorJitter(brightness=0.2, contrast=0.2),    # Light changes
    transforms.RandomAffine(degrees=0, translate=(0.05, 0.05)), # Slight shift
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [11]:
# Define transforms to pre-process our validation data (no augmentation)

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [12]:
# Apply respective transforms to each split

# Wrap subsets with transform override
class TransformDataset(torch.utils.data.Dataset):
    def __init__(self, subset, transform):
        self.subset = subset
        self.transform = transform

    def __len__(self):
        return len(self.subset)

    def __getitem__(self, idx):
        x, y = self.subset[idx]
        return self.transform(x), y

train_dataset = TransformDataset(train_subset, train_transform)
val_dataset = TransformDataset(val_subset, val_transform)

In [13]:
# Set up our data loaders

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

In [14]:
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)  # Binary classification

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 200MB/s]


In [15]:
def train_model(model, train_loader, val_loader, criterion, optimizer, device, epochs=10, save_path='best_model.pth'):
    model.to(device)
    best_val_acc = 0.0  # Keep track of the best validation accuracy

    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")
        print("-" * 30)

        # Training phase
        model.train()
        train_loss = 0.0
        train_correct = 0
        total_train = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            train_correct += (predicted == labels).sum().item()
            total_train += labels.size(0)

        avg_train_loss = train_loss / total_train
        train_acc = train_correct / total_train

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        total_val = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * images.size(0)
                _, predicted = torch.max(outputs, 1)
                val_correct += (predicted == labels).sum().item()
                total_val += labels.size(0)

        avg_val_loss = val_loss / total_val
        val_acc = val_correct / total_val

        # Save model if it's the best so far
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), save_path)
            print(f"New best model saved with val acc: {val_acc*100:.2f}%")

        # Report results
        print(f"Train Loss: {avg_train_loss:.4f} | Train Acc: {train_acc*100:.2f}%")
        print(f"Val Loss: {avg_val_loss:.4f} | Val Acc: {val_acc*100:.2f}%")

In [16]:
train_model(model, train_loader, val_loader, criterion, optimizer, device, epochs=10)


Epoch 1/10
------------------------------
New best model saved with val acc: 78.57%
Train Loss: 0.5499 | Train Acc: 71.15%
Val Loss: 0.4372 | Val Acc: 78.57%

Epoch 2/10
------------------------------
New best model saved with val acc: 85.71%
Train Loss: 0.1454 | Train Acc: 96.15%
Val Loss: 0.2892 | Val Acc: 85.71%

Epoch 3/10
------------------------------
Train Loss: 0.0965 | Train Acc: 96.15%
Val Loss: 0.1691 | Val Acc: 85.71%

Epoch 4/10
------------------------------
New best model saved with val acc: 92.86%
Train Loss: 0.1465 | Train Acc: 92.31%
Val Loss: 0.1178 | Val Acc: 92.86%

Epoch 5/10
------------------------------
New best model saved with val acc: 100.00%
Train Loss: 0.0678 | Train Acc: 96.15%
Val Loss: 0.0631 | Val Acc: 100.00%

Epoch 6/10
------------------------------
Train Loss: 0.0276 | Train Acc: 100.00%
Val Loss: 0.0697 | Val Acc: 100.00%

Epoch 7/10
------------------------------
Train Loss: 0.1039 | Train Acc: 96.15%
Val Loss: 0.1171 | Val Acc: 92.86%

Epoch 8/

In [20]:
uploaded = files.upload()

# Get uploaded image path
image_path = list(uploaded.keys())[0]

# Convert PNG to JPEG if applicable
if image_path.lower().endswith('.png'):
    img = Image.open(image_path).convert('RGB')
    jpg_path = os.path.splitext(image_path)[0] + '.jpg'
    img.save(jpg_path, 'JPEG')
    image_path = jpg_path  # Update path to new JPG

# Define transform (same as val_transform)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Load image
img = Image.open(image_path).convert('RGB')
img_tensor = transform(img).unsqueeze(0)  # Add batch dimension

model.eval()
img_tensor = img_tensor.to(device)

with torch.no_grad():
    outputs = model(img_tensor)
    probs = torch.softmax(outputs, dim=1)
    confidence, pred = torch.max(probs, 1)

# Assuming class 0 = fake, class 1 = real
class_names = full_dataset.classes  # ['fake', 'real']
predicted_class = class_names[pred.item()]
confidence_score = confidence.item() * 100

print(f"Prediction: {predicted_class} ({confidence_score:.2f}% confidence)")

Saving real_02.jpg to real_02.jpg
Prediction: fake (68.03% confidence)
